In [71]:
import pickle 
import os 

graphinfo = pickle.load(open("codegraphs/diversevul/diversevul_graph_info.pkl", "rb"))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f15b45b38d0>>
Traceback (most recent call last):
  File "/opt/conda/envs/pyg/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
commitcounts = {}
for d in graphinfo:
    if d['commit_id'] not in commitcounts:
        commitcounts[d['commit_id']] = 0
    commitcounts[d['commit_id']] += 1

In [14]:
!pip install python-Levenshtein

In [72]:
import Levenshtein as lev

global_store = {}
def load_codegraph(commit_id, hash, target):
    filename = str(commit_id)+'_'+str(hash)+'_'+str(target)+'.c'
    filename = 'codegraphs/diversevul/c_files/' +filename
    # return text file text
    if filename not in global_store:
        global_store[filename] = open(filename, 'r').read()
    return global_store[filename]

# identify all positive files
# for each positive file find the corresponding negative file with smalles levenstein distance

positive_files = []
negative_files = []

from tqdm.auto import tqdm
for d in tqdm(graphinfo):
    filename = str(d['commit_id'])+'_'+str(d['hash'])+'_'+str(d['target'])+'.cpg.pt'
    if d['target'] != 1:
        continue 
    
    
    positive_files.append(filename)
    # find corresponding negative file
    
    file1 = load_codegraph(d['commit_id'], d['hash'], d['target'])
    
    
    
    # print(file1)
    min_dist = float('inf')
    for d2 in graphinfo:
        if d2['commit_id'] == d['commit_id'] and d2['project'] == d['project'] and d2['hash'] != d['hash']:
        
            file2 = load_codegraph(d2['commit_id'], d2['hash'], d2['target'])
            if file2.startswith(file1[:10]) and d2['target']==0:
                dist = lev.distance(file1, file2)
                if dist < min_dist:
                    min_dist = dist
                    min_file = str(d2['commit_id'])+'_'+str(d2['hash'])+'_'+str(d2['target'])+'.cpg.pt'
                    min_file2 = file2
                    
        negative_files.append(min_file)
    
        

  1%|          | 1847/330492 [02:19<6:52:50, 13.27it/s] 


KeyboardInterrupt: 

In [74]:
import Levenshtein as lev
from multiprocessing import Pool
from tqdm.auto import tqdm
import os

def load_codegraph(args):
    global_store, commit_id, hash, target = args
    filename = 'codegraphs/diversevul/c_files/' + str(commit_id) + '_' + str(hash)+ '_' + str(target) + '.c'
    if filename not in global_store:
        with open(filename, 'r') as f:
            global_store[filename] = f.read()
    return global_store[filename]

def find_min_dist(args):
    global_store, d, file1 = args
    min_dist = float('inf')
    min_file = None
    for d2 in graphinfo:
        if d2['commit_id'] == d['commit_id'] and d2['project'] == d['project'] and d2['hash'] != d['hash']:
            file2 = load_codegraph((global_store, d2['commit_id'], d2['hash'], d2['target']))
            if file2.startswith(file1[:10]) and d2['target']==0:
                dist = lev.distance(file1, file2)
                if dist < min_dist:
                    min_dist = dist
                    min_file = str(d2['commit_id']) + '_' + str(d2['hash']) + '_' + str(d2['target']) + '.cpg.pt'
                    min_file2 = file2
    return min_file

positive_files = []
negative_files = []
global_store = {}

pool = Pool(os.cpu_count())
for d in tqdm(graphinfo):
    if d['target'] != 1: continue
    filename = str(d['commit_id']) + '_' + str(d['hash']) + '_' + str(d['target'])+'.cpg.pt'
    positive_files.append(filename)
    file1 = pool.apply_async(load_codegraph, ((global_store, d['commit_id'], d['hash'], d['target']),)).get()
    min_file = pool.apply_async(find_min_dist, ((global_store, d, file1),)).get()
    if min_file: negative_files.append(min_file)
    
pool.close() 
pool.join()

# save to file in codegraph diversevul folder
pickle.dump((positive_files, negative_files), open('codegraphs/diversevul/positive_negative_files.pkl', 'wb'))


  0%|          | 89/330492 [00:04<4:42:17, 19.51it/s]


KeyboardInterrupt: 

In [14]:
import pickle
pos, neg = pickle.load(open('codegraphs/diversevul/positive_negative_files_approxdist.pkl', 'rb'))


In [15]:
len(set(pos)), len(neg)

(18945, 17281)

In [60]:
negative_files

['aab49e934de1fff046e659cbec46e3d053b41c34_317148741482387142957939482611431568327_0.cpg.pt',
 'a10e778bfb7ce9caa1f91666ddf2705db7982d68_202876754177376143376087202132564869711_0.cpg.pt',
 '158d8a6b088662ce9d31e0c777c6ebe90efdc854_49600625714303032256535848130576274680_1.cpg.pt',
 '270a406ac94b5fc5cc9ef59fc61e3b4b95648a3e_146126537554895180438740719158525386157_0.cpg.pt',
 '095cbc48a8f0090f3b0abc6155f2b61943c9eafb_233391808129812884635064530242511031626_0.cpg.pt',
 'afe98b7829d50806559acac9b530acb8283c3bf4_283288526103973642758351842206259716354_0.cpg.pt',
 '188c196d4da60bdde9190d2fc532650d17f7af2d_58278029812532203694375897255278443003_1.cpg.pt',
 '8e76d0404b7f664ee6719fd98f0483f0ac4669d6_156440279598559106269883503657787939006_0.cpg.pt',
 'fcd4b5335a6df4e0676ee32e2267ca71d70fe623_134084765210679426355834754154630655990_1.cpg.pt',
 'fb58e69a84f4fde603a630d2c9df2fa3be16d846_202718344227424850816910741234671335901_0.cpg.pt',
 '74555e7c26b2c61bb8e67b7d6a6f4d2b8eb3a5f3_2955722416961559712

In [11]:
os.listdir('codegraphs/diversevul/c_files')[0]

'b9aea5854bcf6f2de8f7a7f1550874e392b94be2_323732882817984680809054416207973124051_0.c'

In [9]:
commitcounts

{'7ad6162573ba79a4392c63b453ad0220ca6c5ace': 10,
 '055ecbc62878e86287d742c7246c21606cee8183': 27,
 '251fc70e9722f931eec23a34030d05ba5f747b0e': 2,
 'b2eb849d4b1fdb6f35d5c46958c7f703cf64cfef': 90,
 '65d35a09979e63541afc5bfc595b9f1b1b4ae069': 8,
 '822191a2fa1584a29c3224ab328507adcaeac1ab': 66,
 '7ab442d7e0a76402c12553ee256f756097cae2d2': 49,
 'c483bab099cb89e92b7cad94a52fcdaf37e56657': 30,
 '188f83dfe0eeecd1427d0d255cc97dbf7ef6b4b7': 70,
 '29eb51101c02df517ca64ec472d7501127ad1da8': 29,
 '41d5e5d73ecef4ef56b7b4cde962929a712689b4': 141,
 'ccec6e2c4a74adf76ed4e2478091a311b1806212': 23,
 '176df2457ef6207156ca1a40991c54ca01fef567': 2,
 '85237f202d46d55c1bffe0c5b1aa3ddc0f1dce4d': 35,
 'be6aab0e9fa6d3c6d75aa1e38ac972d8b4ee82b8': 48,
 '721151d004dcf01a71b12bb6b893f9160284cf6e': 19,
 '04045f98e0457aba7d4e6736f37eed189c48a5f7': 25,
 '39ebc0276bada8bb70e067cb6d0eb71839c0fb08': 31,
 '4031ca321d8cb5798c316ab39c7a5dc88a61fdd7': 74,
 '0f13864e5b24d9cbe18d125d41bfa4b726a82e40': 68,
 '920fc941a9617f95ccb2

In [8]:
graphinfo

[{'hash': 73008646937836648589283922871188272089,
  'commit_id': '7ad6162573ba79a4392c63b453ad0220ca6c5ace',
  'project': 'gnutls',
  'target': 1,
  'cwe': [],
  'size': 157,
  'message': 'added an extra check while checking the padding.'},
 {'hash': 211824207069112513181516095447837228041,
  'commit_id': '055ecbc62878e86287d742c7246c21606cee8183',
  'project': 'php-src',
  'target': 1,
  'cwe': ['CWE-264'],
  'size': 22,
  'message': 'Improve check for :memory: pseudo-filename in SQlite'},
 {'hash': 21401706257394042943815500829552774160,
  'commit_id': '251fc70e9722f931eec23a34030d05ba5f747b0e',
  'project': 'busybox',
  'target': 1,
  'cwe': [],
  'size': 232,
  'message': 'uncompress: fix buffer underrun by corrupted input\n\nSigned-off-by: Denys Vlasenko <vda.linux@googlemail.com>'},
 {'hash': 135590882627853658533498335902319684573,
  'commit_id': 'b2eb849d4b1fdb6f35d5c46958c7f703cf64cfef',
  'project': 'qemu',
  'target': 1,
  'cwe': ['CWE-787'],
  'size': 66,
  'message': 'CVE-

In [7]:
commitcounts

{'7ad6162573ba79a4392c63b453ad0220ca6c5ace': 10,
 '055ecbc62878e86287d742c7246c21606cee8183': 27,
 '251fc70e9722f931eec23a34030d05ba5f747b0e': 2,
 'b2eb849d4b1fdb6f35d5c46958c7f703cf64cfef': 90,
 '65d35a09979e63541afc5bfc595b9f1b1b4ae069': 8,
 '822191a2fa1584a29c3224ab328507adcaeac1ab': 66,
 '7ab442d7e0a76402c12553ee256f756097cae2d2': 49,
 'c483bab099cb89e92b7cad94a52fcdaf37e56657': 30,
 '188f83dfe0eeecd1427d0d255cc97dbf7ef6b4b7': 70,
 '29eb51101c02df517ca64ec472d7501127ad1da8': 29,
 '41d5e5d73ecef4ef56b7b4cde962929a712689b4': 141,
 'ccec6e2c4a74adf76ed4e2478091a311b1806212': 23,
 '176df2457ef6207156ca1a40991c54ca01fef567': 2,
 '85237f202d46d55c1bffe0c5b1aa3ddc0f1dce4d': 35,
 'be6aab0e9fa6d3c6d75aa1e38ac972d8b4ee82b8': 48,
 '721151d004dcf01a71b12bb6b893f9160284cf6e': 19,
 '04045f98e0457aba7d4e6736f37eed189c48a5f7': 25,
 '39ebc0276bada8bb70e067cb6d0eb71839c0fb08': 31,
 '4031ca321d8cb5798c316ab39c7a5dc88a61fdd7': 74,
 '0f13864e5b24d9cbe18d125d41bfa4b726a82e40': 68,
 '920fc941a9617f95ccb2